In [1]:
import sys
import os
import glob
import json
import matplotlib.pyplot as plt
# BeautifulSoup, for cleaning HTML codes
from bs4 import BeautifulSoup
# Watson Python SDK
#pip install watson_developer_cloud==2.4.1
#pip uninstall watson_developer_cloud
import watson_developer_cloud


path = os.path.dirname(os.path.realpath(r'E:\Important\important files from desktop\NLP.Udacity\IBM Watson project\AIND-NLP-Bookworm-master//helper.py'))
sys.path.append(path)
import helper

#المصادقةAuthentication


In [2]:
#Credentials أوراق اعتماد

    
path = open('E://Important//important files from desktop//NLP.Udacity//IBM Watson project//service-credentials.json')
data = json.load(path)
#print(data)

discovery_creds = helper.fetch_credentials('discovery')

discovery = watson_developer_cloud.DiscoveryV1(
                        version='2018-08-01',
                        url='https://api.us-south.discovery.watson.cloud.ibm.com/instances/ff59d0e6-c27c-4a0f-b76c-0331f6008599',              # it is url only not like the guidline
                        iam_apikey='VJm45LvlWVQmoUoCW9dEvGwZNH4PsWz1JNY11YFLqkv0' )

In [3]:
#Creating Environment

# Prepare an environment to work in
env, env_id = helper.fetch_object(
    discovery, "environment", "Bookworm",
    create=True, create_args=dict(
        description="A space to read and understand stories"  # feel free to edit
    ))

Found environment: Bookworm (a274cd4c-3d93-4525-97bd-7fcc6046d2b5)


In [4]:
#Verify configuration
# Lists existing configurations for the service instance and store default configuration id
configurations = discovery.list_configurations(environment_id=env_id).get_result()
cfg_id =  configurations['configurations'][0]['configuration_id']
#print(json.dumps(configurations, indent=2))

# Get default configuration details
config = discovery.get_configuration(environment_id=env_id, configuration_id=cfg_id).get_result()
#print(json.dumps(config, indent=2))

In [5]:

# Prepare a collection of documents to use
col, col_id = helper.fetch_object(discovery, "collection", "Story Chunks", environment_id=env_id,
    create=True, create_args=dict(
        environment_id=env_id, configuration_id=cfg_id,
        description="Stories and plots split up into chunks suitable for answering")
    )

Found collection: Story Chunks (e1e4e350-2cff-4944-a04c-864afc8ee52b)


In [ ]:
#configuration تهيئة

# Lists existing configurations for the service instance and store default configuration id
configurations = discovery.list_configurations(environment_id=env_id).get_result()
cfg_id =  configurations['configurations'][0]['configuration_id']
#print(json.dumps(configurations, indent=2))

# Get default configuration details
config = discovery.get_configuration(environment_id=env_id, configuration_id=cfg_id).get_result()
#print(json.dumps(config, indent=2))


# Test configuration on some sample text
#splitting paragraphs & Add documents 

data_dir = "data"
doc_ids = []  # to store the generated id for each document added

for filename in glob.glob(os.path.join(data_dir, "Star-Wars", "E://Important//important files from desktop//NLP.Udacity//IBM Watson project//AIND-NLP-Bookworm-master//data//Star-Wars//*.html")):
    print("Adding file:", filename)
    with open(filename, "r") as f:
        # Split each individual <p> into its own "document"
        doc = f.read()
        soup = BeautifulSoup(doc, 'html.parser')
        for i, p in enumerate(soup.find_all('p')):
            doc_info = discovery.add_document(
                environment_id=env_id, 
                collection_id=col_id,
                file=json.dumps({"text": p.get_text(strip=True)}),
                filename='n',
                file_content_type="application/json",
                metadata=json.dumps({"title": soup.title.get_text(strip=True)})
            ).get_result() # add get_result() here

            doc_ids.append(doc_info["document_id"])
print("Total", len(doc_ids), "documents added.")



In [23]:
# Connect to the Assistant service instance
# TODO: Enter your username and password from the Service Credentials tab in service-credentials.json

path = open('E://Important//important files from desktop//NLP.Udacity//IBM Watson project//AIND-NLP-Bookworm-master//service-credentials.json')
data = json.load(path)

assistant_cred = helper.fetch_credentials('assistant')

assistant = watson_developer_cloud.AssistantV1(   # it is AssistantV1 not as the guidline
                        version='2018-08-01',
                        url='https://api.us-south.assistant.watson.cloud.ibm.com/instances/9d87c241-83e3-484f-a14c-31ec9c2c8e78',   # it is url only not like the guidline
                        iam_apikey='VYerhD1AUXmXV133DTn5TlMZxG4NAGKci0pqfEQlkxyZ' )

In [21]:
#wrk, wrk_id = helper.fetch_object(assistant, "workspace", "My first assistant")
wrk= 'My first skill'
wrk_id= 'c9b7a58e-86ed-4c87-9c6a-b2dd726a1775'

In [14]:
# Get all the entities from the collection and group them by type
response = discovery.query(environment_id=env_id, collection_id=col_id,
    query_options={
        "return": "enriched_text.entities.type,enriched_text.entities.text"
    }).get_result()

# Group individual entities by type ("Person", "Location", etc.)
entities_by_type = {}
for document in response["results"]:
    for entity in document["enriched_text"]["entities"]:
        if entity["type"] not in entities_by_type:
            entities_by_type[entity["type"]] = set()
        entities_by_type[entity["type"]].add(entity["text"])

# Ignore case to avoid duplicates
for entity_type in entities_by_type:
    entities_by_type[entity_type] = {
        e.lower(): e for e in entities_by_type[entity_type]
    }.values()

# Restructure for loading into Assistant workspace
entities_grouped = [{
    "entity": entity_type,
    "values": [{"value": entity} for entity in entities]}
        for entity_type, entities in entities_by_type.items()]
entities_grouped



[{'entity': 'Person',
  'values': [{'value': 'Luke'},
   {'value': 'Boss Nass'},
   {'value': 'Obi-Wan'},
   {'value': 'Han Solo'},
   {'value': 'Kylo Ren'},
   {'value': 'Supreme Chancellor Palpatine'},
   {'value': 'Anakin'},
   {'value': 'Queen Amidala'},
   {'value': 'Vader'},
   {'value': 'Mace Windu'},
   {'value': 'Rey'},
   {'value': 'Darth Sidious'},
   {'value': 'Darth Vader'},
   {'value': 'Amidala'},
   {'value': 'PadmÃ©'},
   {'value': 'Poe'},
   {'value': 'Obi-Wan Kenobi'},
   {'value': 'Mustafar'},
   {'value': 'Finn'},
   {'value': 'Senator Palpatine'},
   {'value': 'Viceroy Nute Gunray'},
   {'value': 'Leia'},
   {'value': 'Princess Leia'},
   {'value': 'Count Dooku'},
   {'value': 'Han'},
   {'value': 'Organa'},
   {'value': 'Ren'},
   {'value': 'Qui-Gon'},
   {'value': 'Palpatine'},
   {'value': 'Jango Fett'},
   {'value': 'Beru Lars'},
   {'value': 'Yoda'},
   {'value': 'Polis Massa'},
   {'value': 'Geonosis'},
   {'value': 'Chancellor Valorum'},
   {'value': 'Poe D

In [24]:
# Add these grouped entities to the Assistant workspace
assistant.update_workspace(workspace_id=wrk_id, entities=entities_grouped).get_result()

workspace_details = assistant.get_workspace(workspace_id=wrk_id, export=True).get_result()
print(json.dumps(workspace_details["entities"], indent=2))

[
  {
    "entity": "Company",
    "values": [
      {
        "type": "synonyms",
        "value": "Federation Viceroy Nute Gunray",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "Falcon",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "General Hux",
        "synonyms": []
      }
    ]
  },
  {
    "entity": "Organization",
    "values": [
      {
        "type": "synonyms",
        "value": "Galactic Senate",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "droid army",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "Jedi Council",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "Trade Federation",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "Rebel Alliance",
        "synonyms": []
      },
      {
        "type": "synonyms",
        "value": "Federation",


In [29]:
# TODO: Run a sample question through Assistant service

# Then ask a sample question
question= "Where was Luke born?"
results = assistant.message(workspace_id=wrk_id,
                            input={'text': question}, context=context).get_result()

#print(json.dumps(results, indent=2))

#TODO: Identify the intent(s) the user expressed (typically a single one)
query_intents = [intent["intent"] for intent in results["intents"]]
print("Intent(s):", query_intents)

# TODO: Extract the entities found in the question text
query_entities = [entity["value"] for entity in results["entities"]]
print("Entities:", query_entities)

# TODO: (optional) Find out what dialog node was triggered

# TODO: Query the Discovery service based on the intent and entities
query_results = discovery.query(environment_id=env_id, collection_id=col_id,
    query_options={
        "query": "text:{}".format(",".join("\"{}\"".format(e) for e in query_entities)),
        "return": "text"
    }).get_result()
print(json.dumps(query_results, indent=2))

Intent(s): ['Place']
Entities: ['Luke']
{
  "matching_results": 42,
  "session_token": "1_BbKUC8Txke618qy3_cqS6f8WLa",
  "results": [
    {
      "id": "09436fd0-0cf1-4cef-bcf6-03f22adbf981",
      "result_metadata": {
        "confidence": 0,
        "score": 1
      },
      "text": "Palpatine appoints Anakin to the Jedi Council as his representative, but the Council declines to grant Anakin the rank of Jedi Master and orders him to spy on Palpatine, causing Anakin's faith in the Jedi to diminish significantly. Palpatine tantalizes Anakin with secret knowledge of the dark side of the Force, including the power to save his loved ones from dying. Meanwhile, Obi-Wan travels to the planet Utapau to deal with General Grievous, and Yoda travels to Kashyyyk to defend the planet from invasion. Tempting Anakin, Palpatine eventually reveals that he is the Sith Lord Darth Sidious, saying that only he has the knowledge to save Padm\u00c3\u00a9 from dying. Anakin reports Palpatine's treachery to 